# LOGIN AND FETCH TOKEN

In [3]:
import globus_sdk
import requests
import json

# Globus ID and Scope
# ===================
auth_client_id = "58fdd3bc-e1c3-4ce5-80ea-8d6b87cfb944"

# Define the inference-gateway resource service scope
# This will be publicly available to users
gateway_client_id = "681c10cc-f684-4540-bcd7-0b4df3bc26ef"
gateway_scope = f"https://auth.globus.org/scopes/{gateway_client_id}/action_all"

# Authentication and Access Token
# ===============================

# Start an Auth client with the vLLM scope
auth_client = globus_sdk.NativeAppAuthClient(auth_client_id)
auth_client.oauth2_start_flow(requested_scopes=gateway_scope)

# Authenticate with your Globus account
authorize_url = auth_client.oauth2_get_authorize_url()
print(f"Please go to this URL and login:\n\n{authorize_url}\n")


Please go to this URL and login:

https://auth.globus.org/v2/oauth2/authorize?client_id=58fdd3bc-e1c3-4ce5-80ea-8d6b87cfb944&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2F681c10cc-f684-4540-bcd7-0b4df3bc26ef%2Faction_all&state=_default&response_type=code&code_challenge=tvqMDDhbDhDoHDXhs8LB2DV_ni3IT8JrvtMLtQ0KsOs&code_challenge_method=S256&access_type=online



In [4]:
# Collect access token to vLLM service
auth_code = "CzV2ZLgrvETP9mJ4dL2SE47tT7S8ng"
token_response = auth_client.oauth2_exchange_code_for_tokens(auth_code)
access_token = token_response.by_resource_server[gateway_client_id]["access_token"]

# LIST ENDPOINTS

In [ ]:
# URL to the inference gateway (needs to end with forward slash /)
#url = "https://data-portal-dev-vmw-01.cels.anl.gov/resource_server/polaris/"
import globus_sdk
import requests
import json
url = "http://localhost:8000/resource_server/list-endpoints/"
# Add access token to the headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}
response = requests.get(url, headers=headers, verify=False)
for endpoint in response.json():
    print(endpoint)
          

{'endpoint_url': '/resource_server/polaris/vllm/completions/', 'model_name': 'meta-llama/Meta-Llama-3-8B-Instruct'}
{'endpoint_url': '/resource_server/polaris/vllm/completions/', 'model_name': 'meta-llama/Meta-Llama-3-70B-Instruct'}
{'endpoint_url': '/resource_server/polaris/vllm/completions/', 'model_name': 'mistralai/Mistral-7B-Instruct-v0.3'}
{'endpoint_url': '/resource_server/polaris/llama-cpp/completions/', 'model_name': 'meta-llama-3-8b-instruct'}
{'endpoint_url': '/resource_server/polaris/llama-cpp/completions/', 'model_name': 'mistral-7b-instruct-v03'}
{'endpoint_url': '/resource_server/polaris/llama-cpp/completions/', 'model_name': 'meta-llama-3-70b-instruct'}


## INFERENCE USING VLLM

In [24]:
# URL to the inference gateway (needs to end with forward slash /)
url = "https://data-portal-dev-vmw-01.cels.anl.gov/resource_server/polaris/vllm/completions/"
#url = "http://localhost:8000/resource_server/polaris/vllm/completions/"
# Add access token to the headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Prepare the inference request
data = {
        "model": "meta-llama/Meta-Llama-3-8B-Instruct",
        "temperature": 0.2,
        "max_tokens": 150,
        "messages":[{"role": "user", "content": "List all proteins that interact with RAD51"}],
        'logprobs':True
}

# Convert data into Json51633660

data_json = json.dumps(data)

# Send the post request to the relay server
# Verify=False is temporary since I use a self-signed certificate
response = requests.post(url, data=data_json, headers=headers, verify=False)

# Print inference response
print(response.status_code)
print(response.json())

200
{'server_response': '{\n    "id": "cmpl-d538c498e13b406098e33b535e369aec",\n    "choices": [\n        {\n            "finish_reason": "length",\n            "index": 0,\n            "logprobs": {\n                "content": [\n                    {\n                        "token": "RAD",\n                        "bytes": [\n                            82,\n                            65,\n                            68\n                        ],\n                        "logprob": -0.6931474208831787,\n                        "top_logprobs": []\n                    },\n                    {\n                        "token": "51",\n                        "bytes": [\n                            53,\n                            49\n                        ],\n                        "logprob": 0.0,\n                        "top_logprobs": []\n                    },\n                    {\n                        "token": " is",\n                        "bytes": [\n                 

In [10]:
# URL to the inference gateway (needs to end with forward slash /)
url = "https://data-portal-dev.cels.anl.gov/resource_server/polaris/vllm/completions/"
#url = "http://localhost:8000/resource_server/polaris/vllm/completions/"
# Add access token to the headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Prepare the inference request
data = {
        "model": "meta-llama/Meta-Llama-3-70B-Instruct",
        "temperature": 0.2,
        "max_tokens": 150,
        "messages":[{"role": "user", "content": "List all proteins that interact with RAD51"}]
}

# Convert data into Json51633660

data_json = json.dumps(data)

# Send the post request to the relay server
# Verify=False is temporary since I use a self-signed certificate
response = requests.post(url, data=data_json, headers=headers)

# Print inference response
print(response.status_code)
print(response.json())

200
{'server_response': '{\n    "id": "cmpl-46f51a7a24924b57a3610c59a7805e29",\n    "choices": [\n        {\n            "finish_reason": "length",\n            "index": 0,\n            "logprobs": null,\n            "message": {\n                "content": "What a great question!\\n\\nRAD51 is a key protein involved in homologous recombination, a crucial process for maintaining genome stability. It interacts with numerous proteins to facilitate DNA repair, replication, and recombination. Here\'s a non-exhaustive list of proteins that interact with RAD51:\\n\\n**1. BRCA1 and BRCA2**: Tumor suppressor proteins that interact with RAD51 to facilitate homologous recombination and DNA repair.\\n\\n**2. DMC1**: A meiosis-specific recombinase that interacts with RAD51 to promote homologous recombination during meiosis.\\n\\n**3. XRCC2 and XRCC3**: Proteins involved in homologous recombination that interact with RAD51 to facilitate DNA repair",\n                "role": "assistant",\n          

In [11]:
# URL to the inference gateway (needs to end with forward slash /)
#url = "https://data-portal-dev-vmw-01.cels.anl.gov/resource_server/polaris/"
import time
start_time = time.time()
url = "https://data-portal-dev.cels.anl.gov/resource_server/polaris/vllm/completions/"
# Add access token to the headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Prepare the inference request
data = {
        "model": "mistralai/Mistral-7B-Instruct-v0.3",
        "temperature": 0.2,
        "max_tokens": 150,
        "messages":[{"role": "user", "content": "List all proteins that interact with RAD51"}],
        'logprobs':True
}

# Convert data into Json51633660

data_json = json.dumps(data)

# Send the post request to the relay server
# Verify=False is temporary since I use a self-signed certificate
response = requests.post(url, data=data_json, headers=headers, verify=False)

# Print inference response
print(response.status_code)
print(response.json())
print("Time for globus to submit and respond",time.time()-start_time)

/opt/anaconda3/envs/llama-cpp-python-env/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data-portal-dev.cels.anl.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200
{'server_response': '{\n    "id": "cmpl-c65bed75f50348ff854ca8e0c061e181",\n    "choices": [\n        {\n            "finish_reason": "length",\n            "index": 0,\n            "logprobs": {\n                "content": [\n                    {\n                        "token": " R",\n                        "bytes": [\n                            32,\n                            82\n                        ],\n                        "logprob": 0.0,\n                        "top_logprobs": []\n                    },\n                    {\n                        "token": "AD",\n                        "bytes": [\n                            65,\n                            68\n                        ],\n                        "logprob": 0.0,\n                        "top_logprobs": []\n                    },\n                    {\n                        "token": "5",\n                        "bytes": [\n                            53\n                        ],\n         

## FINDINGS
* Globus adds an average of 6 seconds to the latency


# INFERENCE USING LLAMACPP

In [ ]:
# URL to the inference gateway (needs to end with forward slash /)
#url = "https://data-portal-dev-vmw-01.cels.anl.gov/resource_server/polaris/"
url = "http://localhost:8000/resource_server/polaris/llama-cpp/completions/"
# Add access token to the headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Prepare the inference request
data = {
        "model": "mistral-7B-instruct-v03",
        "temperature": 0.2,
        "prompt": "List all proteins that interact with RAD51",
        #"n_probs":1
}

# Convert data into Json51633660

data_json = json.dumps(data)

# Send the post request to the relay server
# Verify=False is temporary since I use a self-signed certificate
response = requests.post(url, data=data_json, headers=headers, verify=False)

# Print inference response
print(response.status_code)
print(response.json())

In [ ]:
# URL to the inference gateway (needs to end with forward slash /)
#url = "https://data-portal-dev-vmw-01.cels.anl.gov/resource_server/polaris/"
url = "http://localhost:8000/resource_server/polaris/llama-cpp/completions/"
# Add access token to the headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Prepare the inference request
data = {
        "model": "meta-llama-3-70b-instruct",
        "temperature": 0.2,
        "max_tokens": 150,
        "prompt": "List all proteins that interact with RAD51"    
}

# Convert data into Json51633660

data_json = json.dumps(data)

# Send the post request to the relay server
# Verify=False is temporary since I use a self-signed certificate
response = requests.post(url, data=data_json, headers=headers, verify=False)

# Print inference response
print(response.status_code)
print(response.json())